# Sign Language Alphabets
Sign language is a visual means of communicating through hand signals, gestures, facial expressions, and body language. It’s the main form of communication for the Deaf and Hard-of-Hearing community, but sign language can be useful for other groups of people as well. People with disabilities including Autism, Apraxia of speech, Cerebral Palsy, and Down Syndrome may also find sign language beneficial for communicating.

## Basics of Alphabets and Fingerspelling
Most people start their sign language journey by learning the A-Z or alphabet equivalent in sign form.

The use of the hands to represent individual letters of a written alphabet is called ‘fingerspelling’. It’s an important tool that helps signers manually spell out names of people, places and things that don’t have an established sign.

For example, most sign languages have a specific sign for the word tree, but may not have a specific sign for oak, so o-a-k would be finger spelled to convey that specific meaning.

Of course, not every language uses the Latin alphabet like English, so their sign language alphabet differs as well. Some manual alphabets are one-handed, such as in ASL and French Sign Language, and others use two-hands, like BSL or Auslan. Though there are similarities between some of the different manual alphabets, each sign language has its own style and modifications, and remains unique.

## American Sign Language (ASL)
Although ASL has the same alphabet as English, ASL is not a subset of the English language. American Sign Language was created independently and it has its own linguistic structure. 
Signs are also not expressed in the same order as words are in English. This is due to the unique grammar and visual nature of the sign language. ASL is used by roughly half a million people in the USA.

![Image](https://www.ai-media.tv/wp-content/uploads/2020/09/ASL_Alphabet.jpg)

In [ ]:
# Loading the packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import tensorflow as tf

In [ ]:
# Loading the data
train = pd.read_csv("../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv")
test = pd.read_csv("../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv")

In [ ]:
train.head(10)

In [ ]:
train.describe().T

In [ ]:
# Plotting for the label having more than 1200 rows.
sns.set_style("whitegrid")
plt.figure(figsize=(14,7))
g = sns.countplot(train['label'], color='lightgrey')

for p in g.patches:
    g.annotate('{:.0f}'.format(p.get_height()), (p.get_x(), p.get_height()+10))
    if p.get_height() > 1200:
        p.set_color('blue')
g.set_ylabel('')    
g.set_xlabel('')
g.axes.get_yaxis().set_visible(False)

In [ ]:
# Data splitting and PreProcessing 
training_images = train.iloc[:,1:].values
training_labels = train.iloc[:,0].values

testing_images = test.iloc[:,1:].values
testing_labels = test.iloc[:,0].values

training_images = training_images.reshape(-1,28,28,1)
testing_images = testing_images.reshape(-1,28,28,1)

In [ ]:
# Plotting the first 10 images 
fig, ax = plt.subplots(2,5) 
fig.set_size_inches(10, 10)
k = 0
for i in range(2):
    for j in range(5):
        ax[i,j].imshow(training_images[k].reshape(28, 28) , cmap = "gray")
        k += 1
    plt.tight_layout()

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 rotation_range=40,
                                 fill_mode='nearest',
                                 zoom_range=0.2,
                                 shear_range=0.2,
                                 horizontal_flip=True)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
# Model Building for the data

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import RMSprop

model = Sequential([
    Conv2D(64, (3,3), activation='relu', input_shape=(28,28,1)),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(25,activation='softmax')
])

model.compile(optimizer = 'adam',
              loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.summary()

In [ ]:
train_generator = train_datagen.flow(training_images, training_labels)
valid_generator = validation_datagen.flow(testing_images, testing_labels)

history = model.fit(train_generator,
                    epochs=15,
                    verbose=1,
                   validation_data = valid_generator)

model.evaluate(testing_images, testing_labels, verbose=0)

* Here, we are getting an **accuracy - 72%** for the model evaluation.

In [ ]:
# Plot the chart for accuracy and loss on both training and validation
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
# Predictions 
predictions = model.predict_classes(testing_images)
for i in range(len(predictions)):
    if(predictions[i] >= 9):
        predictions[i] += 1
predictions[:5]   

In [ ]:
# Precision, recall, f1-score for all the classes
from sklearn.metrics import classification_report, confusion_matrix

classes = ["Class " + str(i) for i in range(26) if i != 9]
print(classification_report(test['label'], predictions, target_names = classes))

In [ ]:
# Confusion Matrix for the model predictions
cm = confusion_matrix(test['label'],predictions)

plt.figure(figsize=(12,7))
g = sns.heatmap(cm, cmap='Reds',annot=True,
           fmt='')

* Here, the predictions have some wrong predictions.
* Highest predictions - 12 Label